In [1]:
import torch
import numpy as np
import polars as pl
import itertools
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
class LabelledTensor:
    """
    Classe enveloppe pour un tenseur PyTorch avec des étiquettes explicites pour chaque dimension.

    Attributs
    ---------
    tensor : torch.Tensor
        Le tenseur brut.
    dim_labels : list[str]
        Liste ordonnée des noms des dimensions.
    index_to_label : dict[str, list[str]]
        Dictionnaire associant à chaque nom de dimension la liste de ses étiquettes.
    """

    def __init__(self, tensor: torch.Tensor, dim_labels: list[str], index_to_label: dict[str, list[str]]):
        """
        Initialise un objet LabelledTensor.

        Paramètres
        ----------
        tensor : torch.Tensor
            Le tenseur PyTorch à encapsuler.
        dim_labels : list[str]
            Les noms des dimensions du tenseur.
        index_to_label : dict[str, list[str]]
            Les étiquettes associées à chaque dimension.
        """
        self.tensor = tensor
        self.dim_labels = dim_labels
        self.index_to_label = index_to_label

    def to(self, device):
        """
        Déplace le tenseur vers le périphérique spécifié (CPU ou GPU).

        Paramètre
        ---------
        device : str
            'cpu' ou 'cuda' (ou tout autre périphérique reconnu par PyTorch).
        
        Retourne
        --------
        self : LabelledTensor
            L'objet lui-même après déplacement.
        """
        self.tensor = self.tensor.to(device)
        return self

    def __repr__(self):
        """
        Représentation courte de l'objet pour un affichage rapide.
        """
        return f"LabelledTensor(shape={self.tensor.shape}, dims={self.dim_labels})"

    def display(self, max_elements=100) -> pl.DataFrame:
        """
        Affiche une vue lisible du tenseur sous forme d'un DataFrame Polars avec les étiquettes.

        Paramètres
        ----------
        max_elements : int
            Nombre maximal d'éléments à afficher (pour éviter une surcharge visuelle).

        Retourne
        --------
        pl.DataFrame
            Un DataFrame Polars contenant les valeurs du tenseur et leurs étiquettes.
        """
        dims = self.tensor.shape
        dim_labels = self.dim_labels
        idx_labels = self.index_to_label

        total = self.tensor.numel()
        if total > max_elements:
            print(f"[INFO] Tenseur trop grand ({total} éléments). Affichage des {max_elements} premiers éléments seulement.\n")

        # Création de toutes les combinaisons possibles d’indices
        indices = list(itertools.product(*[range(d) for d in dims]))
        records = []

        for idx in indices[:max_elements]:
            labels = [idx_labels[dim_labels[i]][idx[i]] for i in range(len(idx))]
            value = self.tensor[idx].item()
            records.append((*labels, value))

        columns = dim_labels + ["valeur"]
        df = pl.DataFrame(records, schema=columns)
        
        return df
    
    def to_dataframe( self, index_dim: str, column_dim: str | None = None, 
                     index_name: str | None = None, value_name: str = "valeur") -> pl.DataFrame:
        """
        Convertit le tenseur en DataFrame Polars avec étiquettes.

        - Si le tenseur est 2D, effectue un pivot large.
        - Si 1D, affiche simplement index + valeur.

        Paramètres
        ----------
        index_dim : str
            Nom de la dimension pour les lignes.
        column_dim : str, optional
            Nom de la dimension pour les colonnes (pour tenseurs 2D).
        index_name : str, optional
            Nom de la colonne d’index. Si None, utilise index_dim.
        value_name : str
            Nom de la colonne des valeurs (pour tenseurs 1D).

        Retourne
        --------
        pl.DataFrame
            DataFrame avec index et colonnes ou valeurs.
        """
        if index_dim not in self.dim_labels:
            raise ValueError(f"{index_dim} n’est pas une dimension valide.")

        dim_index = self.dim_labels.index(index_dim)

        if self.tensor.ndim == 1:
            if column_dim is not None:
                raise ValueError("column_dim doit être None pour les tenseurs 1D.")
            labels = self.index_to_label[index_dim]
            values = self.tensor.cpu().numpy().tolist()
            return pl.DataFrame({
                index_name or index_dim: labels,
                value_name: values
            })

        elif self.tensor.ndim == 2:
            if column_dim is None or column_dim not in self.dim_labels:
                raise ValueError("column_dim est requis pour les tenseurs 2D.")
            row_idx = self.dim_labels.index(index_dim)
            col_idx = self.dim_labels.index(column_dim)

            if (row_idx, col_idx) != (0, 1):
                tensor = self.tensor.permute(row_idx, col_idx)
            else:
                tensor = self.tensor

            row_labels = self.index_to_label[index_dim]
            col_labels = self.index_to_label[column_dim]
            values_np = tensor.cpu().numpy()

            df_dict = {index_name or index_dim: row_labels}
            for j, col in enumerate(col_labels):
                df_dict[col] = values_np[:, j]

            return pl.DataFrame(df_dict)

        else:
            raise ValueError("Seuls les tenseurs 1D ou 2D sont pris en charge.")

In [4]:
def create_symmetric_matrix(df: pl.DataFrame, device="cpu") -> LabelledTensor:
    """
    Construit une matrice symétrique des temps de parcours à partir d’un DataFrame triangle supérieur.

    Paramètres :
    ------------
    df : pl.DataFrame
        Doit contenir les colonnes "Idloc_start", "Idloc_end", "temps_parcours".
    device : str
        'cpu' ou 'cuda' selon l’appareil souhaité.

    Retour :
    --------
    LabelledTensor
        Matrice [i, j] symétrique, avec labels.
    """
    # Étape 1 : identifiants uniques ordonnés
    unique_locs = pl.concat([df["Idloc_start"], df["Idloc_end"]]).unique().sort()
    idx_to_id = unique_locs.to_list()
    id_to_idx = {idloc: idx for idx, idloc in enumerate(idx_to_id)}
    n = len(idx_to_id)

    # Étape 2 : conversion en indices numpy
    i_idx = df["Idloc_start"].to_numpy()
    j_idx = df["Idloc_end"].to_numpy()
    values = df["temps_parcours"].to_numpy()

    # Étape 3 : mapping des ID vers index
    i_indices = np.vectorize(id_to_idx.get)(i_idx)
    j_indices = np.vectorize(id_to_idx.get)(j_idx)

    # Étape 4 : remplissage de la matrice via vecteurs
    T = torch.full((n, n), float("inf"), device=device)
    indices = torch.tensor(np.stack([i_indices, j_indices]), device=device)
    distances = torch.tensor(values, dtype=torch.float32, device=device)

    # Triangle supérieur
    T[indices[0], indices[1]] = distances
    # Symétrie
    T[indices[1], indices[0]] = distances
    # Diagonale
    T.fill_diagonal_(0.0)

    return LabelledTensor(T, ["i", "j"], {"i": idx_to_id, "j": idx_to_id})

In [5]:
def create_population_tensor(df_pop: pl.DataFrame, idloc_order: list[str], device="cpu") -> LabelledTensor:
    """
    Crée un vecteur de population ordonné selon idloc.

    Paramètres :
    ------------
    df_pop : pl.DataFrame 
        Contient les colonnes "idloc" et "taille_population".
    idloc_order : list[str]
        Ordre des localités à respecter.
    device : str
        Appareil.

    Retour :
    --------
    LabelledTensor
        Vecteur [i] avec les tailles de population.
    """
    pop = torch.tensor(
        [df_pop.filter(pl.col("Idloc") == loc)["taille_population"][0] for loc in idloc_order],
        dtype=torch.float32,
        device=device
    )
    return LabelledTensor(pop, ["i"], {"i": idloc_order})

In [6]:
def create_infrastructure_tensor(df: pl.DataFrame, device="cpu") -> LabelledTensor:
    """
    Construit un tenseur D[i, t] représentant les infrastructures disponibles.

    Les noms des colonnes encodent le secteur (k) dans leurs deux premiers caractères.

    Paramètres :
    ------------
    df : pl.DataFrame
        Doit contenir la colonne "idloc" et des colonnes de sous-secteurs.
    device : str
        Appareil.

    Retour :
    --------
    LabelledTensor
        Tenseur [i, t] : localité × sous-secteur.
    """
    idlocs = df["idloc"].to_list()
    df_data = df.drop("idloc")
    sous_secteurs = df_data.columns

    D = torch.tensor(
        df_data.to_numpy(),
        dtype=torch.float32,
        device=device
    )

    return LabelledTensor(D, ["i", "t"], {
        "i": idlocs,
        "t": sous_secteurs
    })

In [7]:
def compute_Y(T: LabelledTensor, D: LabelledTensor) -> LabelledTensor:
    """
    Calcule le temps minimal de chaque localité i vers une autre j disposant d'une infrastructure t.

    Y[i, t] = min_j { T[i, j] | D[j, t] > 0 }

    Version sans boucle explicite, entièrement vectorisée.

    Paramètres :
    ------------
    T : LabelledTensor
        Matrice des temps de parcours [i, j].
    D : LabelledTensor
        Tenseur d'infrastructure [j, t].

    Retour :
    --------
    LabelledTensor
        Tenseur [i, t] contenant les temps minimaux vers une infrastructure.
    """
    T_tensor = T.tensor  # [i, j]
    D_tensor = D.tensor.bool()  # [j, t]

    i_size, j_size = T_tensor.shape
    t_size = D_tensor.shape[1]

    # Stocke les résultats pour chaque t
    results = []

    for t in range(t_size):
        # Indices des j où l'infrastructure t existe
        j_mask = D_tensor[:, t]  # [j]
        if j_mask.any():
            # Sous-matrice T[:, j sélectionnés]
            T_filtered = T_tensor[:, j_mask]
            min_t = T_filtered.min(dim=1).values  # [i]
        else:
            # Aucun j valide pour ce t
            min_t = torch.full((i_size,), float("inf"), device=T_tensor.device)

        results.append(min_t)

    # Concatène les résultats pour obtenir [i, t]
    Y_tensor = torch.stack(results, dim=1)  # [i, t]

    return LabelledTensor(Y_tensor, ["i", "t"], {
        "i": T.index_to_label["i"],
        "t": D.index_to_label["t"]
    })

In [9]:
def compute_Y_agg(Y: LabelledTensor, P: LabelledTensor, exclude_zero: bool = False) -> LabelledTensor:
    """
    Moyenne pondérée de Y[i, t] par la population de chaque localité i.

    Retourne Y_[t]

    Paramètres
    ----------
    Y : LabelledTensor
        Tenseur Y[i, t]
    P : LabelledTensor
        Population par localité [i]
    exclude_zero : bool
        Si True, exclut les lignes où Y[i, t] == 0 du calcul pour chaque t.

    Retour
    ------
    LabelledTensor
        Moyenne pondérée [t]
    """
    Y_tensor = Y.tensor  # [i, t]
    P_tensor = P.tensor.view(-1, 1)  # [i, 1]

    if exclude_zero:
        mask = Y_tensor != 0  # [i, t]
        weighted = Y_tensor * P_tensor * mask  # zero quand Y == 0
        weights = P_tensor * mask
    else:
        weighted = Y_tensor * P_tensor
        weights = P_tensor.expand_as(Y_tensor)

    Y_mean = weighted.sum(dim=0) / weights.sum(dim=0).clamp(min=1e-8)

    return LabelledTensor(Y_mean, ["t"], {
        "t": Y.index_to_label["t"]
    })

In [10]:
def normalize_Y(Y: LabelledTensor, Y_mean: LabelledTensor) -> LabelledTensor:
    """
    Normalise Y[i, t] en divisant chaque élément par la moyenne agrégée Ȳ[t].

    Paramètres :
    ------------
    Y : LabelledTensor
        Tenseur original [i, t].
    Y_mean : LabelledTensor
        Moyenne agrégée [t].

    Retour :
    --------
    LabelledTensor
        Tenseur normalisé [i, t].
    """
    norm_tensor = Y.tensor / Y_mean.tensor.unsqueeze(0) # [i, t] / [1, t]
    return LabelledTensor(norm_tensor, Y.dim_labels, Y.index_to_label)


In [11]:
def clamp_Y(Y: LabelledTensor, max_value=3.0) -> LabelledTensor:
    """
    Tronque les valeurs de Y[i, t] à une valeur maximale.

    Paramètres :
    ------------
    Y : LabelledTensor
        Tenseur à borner.
    max_value : float
        Valeur max autorisée.

    Retour :
    --------
    LabelledTensor
        Tenseur borné.
    """
    clamped = torch.clamp(Y.tensor, max=max_value)
    return LabelledTensor(clamped, Y.dim_labels, Y.index_to_label)


In [12]:
def compute_remoteness_tensor(Mat_dist: LabelledTensor,
                                 Mat_infra: LabelledTensor,
                                 Mat_pop: LabelledTensor,
                                 clamp_max: float = 3.0) -> LabelledTensor:
    """
    Calcule le tenseur d'accessibilité final Y[i, t] normalisé et borné.

    Étapes :
    -------
    1. Calcul des temps minimaux Y[i, t] depuis chaque localité i vers une infrastructure de type t.
    2. Agrégation pondérée par la population pour obtenir Y_mean[t].
    3. Normalisation du tenseur Y[i, t] par Y_mean[t].
    4. Borne la valeur maximale de Y[i, t] à `clamp_max`.

    Paramètres :
    ------------
    Mat_dist : LabelledTensor
        Matrice des distances [i, j].

    Mat_infra : LabelledTensor
        Tenseur d'infrastructure [i, t], booléen ou réel.

    Mat_pop : LabelledTensor
        Vecteur des populations [i].

    clamp_max : float
        Valeur maximale autorisée pour la normalisation (clipping final).

    Retour :
    --------
    LabelledTensor
        Tenseur d'accessibilité [i, t] normalisé et borné.
    """
    Mat_Y = compute_Y(Mat_dist, Mat_infra)
    Mat_Y_mean = compute_Y_agg(Mat_Y, Mat_pop, exclude_zero=True)
    Mat_Y_norm = normalize_Y(Mat_Y, Mat_Y_mean)
    Mat_Y_final = clamp_Y(Mat_Y_norm, max_value=clamp_max)

    return Mat_Y_final


In [13]:
def load_all_matrices(path_dt: str,
                        path_infra: str,
                        path_pop: str,
                        device: str = "cuda") -> tuple[LabelledTensor, LabelledTensor, LabelledTensor]:
    """
    Charge et construit les matrices LabelledTensor nécessaires au calcul d'accessibilité :
    - Matrice des distances symétrique [i, j]
    - Tenseur des infrastructures [i, t]
    - Vecteur de population [i]

    Paramètres :
    ------------
    path_dt : str
        Chemin vers le fichier Parquet contenant les temps de parcours (triangle supérieur).
    path_infra : str
        Chemin vers le fichier Parquet contenant les infrastructures.
    path_pop : str
        Chemin vers le fichier Parquet contenant les populations.
    device : str
        Périphérique ("cpu" ou "cuda").

    Retour :
    --------
    tuple[LabelledTensor, LabelledTensor, LabelledTensor]
        Mat_dist  : Matrice des temps de parcours [i, j]
        Mat_infra : Tenseur des infrastructures [i, t]
        Mat_pop   : Vecteur des populations [i]
    """
    # Lecture des fichiers parquet
    df_polars = pl.read_parquet(path_dt)
    df_infra = pl.read_parquet(path_infra)
    df_pop = pl.read_parquet(path_pop)

    # Construction des matrices
    Mat_dist = create_symmetric_matrix(df_polars, device=device)
    Mat_infra = create_infrastructure_tensor(df_infra, device=device)
    Mat_pop = create_population_tensor(df_pop, df_pop["Idloc"].to_list(), device=device)

    return Mat_dist, Mat_infra, Mat_pop

In [14]:
import polars as pl

def code_decoder(chemin_parquet: str, code: str):
    """
    Décode un code hiérarchique (2, 5 ou 9 chiffres) en labels descriptifs.

    Paramètres
    ----------
    chemin_parquet : str
        Chemin vers le fichier .parquet des codes hiérarchiques.
    code : str
        Code à décoder (2, 5 ou 9 chiffres).

    Retourne
    --------
    tuple
        (dernier label, identifiant normalisé, dictionnaire des niveaux)
    """
    df = pl.read_parquet(chemin_parquet)

    # Normalisation des colonnes
    df = df.with_columns([
        pl.col("code").cast(str).str.strip_chars().str.zfill(2),
        pl.col("id_secteur").cast(str).str.strip_chars().str.zfill(3),
        pl.col("id_Hiera").cast(str).str.strip_chars().str.zfill(4)
    ])

    code_str = str(code).strip()
    n = len(code_str)

    if n == 2:
        match = df.filter(pl.col("code") == code_str)
        if match.height > 0:
            secteur = match[0, "Nom du secteur"]
            return (
                secteur,
                secteur.lower().replace(" ", "_"),
                {"secteur": secteur}
            )
        return ("Code non trouvé", {})

    elif n == 5:
        match = df.filter(
            (pl.col("code") == code_str[:2]) &
            (pl.col("id_secteur") == code_str[2:])
        )
        if match.height > 0:
            secteur = match[0, "Nom du secteur"]
            sous = match[0, "Sous secteur"]
            return (
                sous,
                f"{secteur}--{sous}".lower().replace(" ", "_"),
                {"secteur": secteur, "Sous secteur": sous}
            )
        return ("Code non trouvé", {})

    elif n == 9:
        match = df.filter(
            (pl.col("code") == code_str[:2]) &
            (pl.col("id_secteur") == code_str[2:5]) &
            (pl.col("id_Hiera") == code_str[5:])
        )
        if match.height > 0:
            secteur = match[0, "Nom du secteur"]
            sous = match[0, "Sous secteur"]
            desc = match[0, "Description"]
            return (
                desc,
                f"{secteur}--{sous}--{desc}".lower().replace(" ", "_"),
                {
                    "secteur": secteur,
                    "Sous secteur": sous,
                    "Description": desc
                }
            )
        return ("Code non trouvé", {})

    return ("Le nombre de chiffres du code est imparfait", {})


In [15]:
path_dico = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\dictionnaires_secteurs.parquet"

In [16]:
code_decoder(path_dico, "020030005")

('Code non trouvé', {})

In [17]:
# path_dt = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\dt_matrix_terrain.parquet"
# path_infra = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\infrastructure_matrix_VF.parquet"
# path_pop = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\population_matrix.parquet"

In [18]:
path_dt = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\dt_matrix_ligne_VF.parquet"
path_infra = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\infrastructure_matrix_VF.parquet"
path_pop = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\population_matrix.parquet"

In [19]:
Mat_dist, Mat_infra, Mat_pop = load_all_matrices(path_dt, path_infra, path_pop, device="cuda")

In [20]:
Mat_Y = compute_remoteness_tensor(Mat_dist, Mat_infra, Mat_pop, clamp_max=3.0)

In [21]:
Mat_Y.to_dataframe(index_dim="i", column_dim="t", index_name="Idloc")

Idloc,01,01001,010010001,010010002,01002,010020001,010020002,01003,010030001,010030002,010030003,02,02001,020010001,020010002,020010003,020010004,020010005,02002,020020001,020020002,020020003,02003,020030001,020030002,020030003,020030004,02004,020040003,020040005,03,03001,030010001,03002,030020002,030020003,…,14,14001,140010001,14002,140020001,140020002,14003,140030001,14004,140040001,140040002,140040003,14005,140050001,140050002,140050003,140050004,140050005,16,16001,160010001,16002,160020001,160020002,160020003,160020004,160020005,160020006,16004,160040001,160040002,16005,160050001,160050002,16006,160060001,160060002
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""010020201001""",0.129302,0.247799,0.247799,0.063379,0.073691,0.070797,0.410424,0.114287,1.087008,1.306398,0.008987,0.0,0.0,0.245572,0.161259,0.036313,0.0,0.146319,0.218519,0.127828,0.147678,0.029028,0.0,0.100067,0.04143,0.0,0.036501,0.290996,0.378448,0.118042,0.0,0.0,0.0,0.034729,0.03252,0.205516,…,0.074036,0.072469,0.072469,0.027413,0.097385,0.026249,0.013622,0.013622,0.30007,0.286282,0.209934,0.116444,0.256994,0.161702,0.777942,0.874683,0.266273,0.152794,0.432104,0.260886,0.260886,0.429562,0.703717,0.369973,0.697956,0.151338,0.788904,0.045783,0.043019,1.402566,0.042391,0.092894,0.09272,0.161432,0.432067,1.29385,0.23442
"""010020201003""",0.928523,0.465112,0.465112,0.118961,0.712155,0.68418,0.770354,0.820702,0.75217,1.143981,0.08685,0.0,0.0,0.0,0.288249,0.260765,0.520155,1.050721,0.0,0.91794,0.0,0.280528,0.74134,0.216773,0.400382,0.645595,0.262117,0.546191,0.506075,0.221562,0.0,0.0,0.0,0.249387,0.23353,0.385748,…,0.132339,0.129537,0.129537,0.09502,0.159259,0.090986,0.131639,0.131639,0.436254,0.416208,0.30521,0.16929,0.48237,0.30351,0.752339,0.836734,0.387119,0.204816,0.772381,0.38307,0.38307,0.767838,1.524449,0.199499,1.191877,0.270514,0.754264,0.442448,0.172383,1.426105,0.169866,0.667073,0.665827,0.208291,0.378351,1.268168,0.205276
"""010020201004""",0.736158,0.310815,0.310815,0.079497,0.419549,0.403068,0.514794,0.650674,0.675343,0.976989,0.051166,0.0,0.0,0.0,0.440883,0.206742,0.309944,0.626092,0.0,0.727768,0.0,0.165266,0.41797,0.194631,0.235876,0.363989,0.207813,0.364996,0.432201,0.14806,0.0,0.0,0.0,0.197721,0.185149,0.25303,…,0.42151,0.412587,0.412587,0.062911,0.136011,0.06024,0.077552,0.077552,0.382008,0.364455,0.267259,0.14824,0.322347,0.202823,0.726015,0.797716,0.338982,0.184094,1.848955,0.334401,0.334401,1.83808,1.312548,0.831463,0.426691,0.647568,0.718647,0.260657,0.120854,1.402166,0.119089,0.528874,0.527885,0.136515,0.323121,1.241764,0.175311
"""010020201005""",0.382646,0.251213,0.251213,0.064252,0.218077,0.20951,0.416077,0.338213,0.388973,0.838082,0.026595,0.0,0.0,0.0,0.0,0.107462,0.111908,0.226057,0.0,0.378285,0.0,0.085904,0.135724,0.112101,0.122605,0.118195,0.108019,0.295004,0.370751,0.119668,0.0,0.0,0.0,0.102773,0.096238,0.203598,…,0.219096,0.214458,0.214458,0.044368,0.116673,0.042484,0.040311,0.040311,0.344657,0.32882,0.241128,0.133746,0.260534,0.16393,0.704119,0.76526,0.305839,0.169826,0.667583,0.300889,0.300889,0.663657,0.732153,0.526325,0.240046,0.233811,0.689021,0.135487,0.085373,1.393271,0.084127,0.274902,0.274389,0.109846,0.27718,1.2198,0.150385
"""010020201007""",0.478097,0.339706,0.339706,0.086886,0.272476,0.261772,0.562647,0.42258,0.387293,0.662855,0.063076,0.0,0.668144,0.524092,0.556162,0.208356,0.267828,0.541017,0.0,0.733451,0.0,0.203737,0.0,0.173205,0.153189,0.0,0.209436,0.333996,0.293234,0.161823,0.0,0.0,0.0,0.199265,0.186595,0.235885,…,0.27375,0.267954,0.267954,0.06688,0.092279,0.06404,0.050366,0.050366,0.400114,0.381728,0.279926,0.155266,0.352311,0.221676,0.676496,0.724318,0.355049,0.191011,0.0,0.350645,0.350645,0.0,0.0,0.412675,0.72951,0.559574,0.651649,0.169284,0.138053,1.40407,0.13603

In [81]:
# Récupérer le DataFrame
df = Mat_Y.to_dataframe(index_dim="i", column_dim="t", index_name="Idloc")

In [70]:
decoded_dict = {col: code_decoder(path_dico, col)[1] for col in df.columns if col != "Idloc"}

In [71]:
# Renommer les colonnes de df à l'aide de decoded_dict
col_map = {col: decoded_dict.get(col, col) for col in df.columns}

In [76]:
col_map = {
    k: (k if v == {} else v) for k, v in col_map.items()
}

In [77]:
col_map

{'Idloc': 'Idloc',
 '01': 'administration',
 '01001': 'administration--ambassade',
 '010010001': 'administration--ambassade--consulat',
 '010010002': 'administration--ambassade--ambassade',
 '01002': 'administration--décentralisé',
 '010020001': 'administration--décentralisé--mairie',
 '010020002': 'administration--décentralisé--conseil_régional',
 '01003': 'administration--déconcentré',
 '010030001': 'administration--déconcentré--sous-préfecture',
 '010030002': 'administration--déconcentré--préfecture',
 '010030003': 'administration--déconcentré--district_',
 '02': 'commerce',
 '02001': 'commerce--vente_de_gros_et_détails',
 '020010001': 'commerce--vente_de_gros_et_détails--boutique',
 '020010002': 'commerce--vente_de_gros_et_détails--librairie',
 '020010003': 'commerce--vente_de_gros_et_détails--boulangerie',
 '020010004': 'commerce--vente_de_gros_et_détails--superette/super_marché_',
 '020010005': 'commerce--vente_de_gros_et_détails--centre_commercial',
 '02002': 'commerce--marché',

In [83]:
df = df.rename(col_map)

In [86]:
df.write_csv(r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Output\Indice_eloignement.csv")

In [87]:
Mat_Y1 = compute_Y(Mat_dist, Mat_infra)

In [90]:
Mat_Y1.to_dataframe(index_dim="i", column_dim="t", index_name="Idloc").rename(col_map).write_csv(r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Output\Temps_minimum.csv")

In [91]:
Mat_Y_mean = compute_Y_agg(Mat_Y1, Mat_pop, exclude_zero=True)

In [92]:
Mat_Y_mean.to_dataframe(index_dim="t", index_name="Id_Secteur", value_name="Moyenne")

Id_Secteur,Moyenne
str,f64
"""01""",25611.734375
"""01001""",153036.21875
"""010010001""",153036.21875
"""010010002""",582128.25
"""01002""",44676.1875
…,…
"""160050001""",35693.929688
"""160050002""",340028.71875
"""16006""",199783.71875


In [93]:
# Convert LabelledTensor to DataFrame
df = Mat_Y_mean.to_dataframe(index_dim="t", index_name="Id_Secteur", value_name="Moyenne")

# Create a mapping expression using `replace`
df_renamed = df.with_columns([
    pl.col("Id_Secteur").replace(col_map).alias("Id_Secteur")
])

In [95]:
df_renamed.write_csv(r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Output\Temps_parcours_minimum_moyen_national.csv")